In [51]:
from __future__ import print_function
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import numpy as np
import pandas as pd

from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import fetch_20newsgroups
from sklearn.datasets.twenty_newsgroups import strip_newsgroup_footer
from sklearn.datasets.twenty_newsgroups import strip_newsgroup_quoting
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split


class ArrayCaster(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, data):
        print(data.shape)
        print(np.transpose(np.matrix(data)).shape)
        return np.transpose(np.matrix(data))

union =FeatureUnion([('comment', Pipeline([
            ('selector', ItemSelector(key='comment')),
            #('vect', CountVectorizer(ngram_range=(1,1), binary=True, min_df=3)),
            ('tfidf', TfidfVectorizer())
          ])
        ),

        ('compound', Pipeline([
            ('selector', ItemSelector(key='compound')),
            ('caster', ArrayCaster())
          ])
        )])

svm = SVC(kernel="linear")

pipeline = Pipeline([("features", union), ("svm", svm)])

data_vader = pd.read_csv("train-balanced-sarcasm-vader.csv")


X= data_vader[["comment","compound"]]

print(X)


X_train, X_test, y_train, y_test = train_test_split(X, data_vader["label"],test_size =0.3, random_state=50)
print(X_train.shape)
print(y_train.shape)
print(union)
pipeline.fit(X_train,y_train)
y = pipeline.predict(X_test)
report = classification_report(y_test, y, digits = 3)
print(report)

                                                 comment  compound
0                                             NC and NH.    0.6037
1      You do know west teams play against west teams...    0.3400
2      They were underdogs earlier today, but since G...    0.0000
3      This meme isn't funny none of the "new york ni...   -0.3412
4                        I could use one of those tools.    0.0000
...                                                  ...       ...
19995              Guess this means CF has a conscience.    0.0000
19996  If I've been waiting this long I'm damn sure g...   -0.1759
19997  Surely this will persuade the undecided voters...    0.3164
19998  Yep, because American OPs always post in r/den...    0.2960
19999        Trump supporters are the pick of the litter    0.2732

[20000 rows x 2 columns]
(14000, 2)
(14000,)
FeatureUnion(n_jobs=None,
             transformer_list=[('comment',
                                Pipeline(memory=None,
                           